# Opal Masterclass mit maschinellem Lernen

Lade Funktionen und alles was sonst noch nötig ist...

In [ ]:
import os
if "opal_mc_ml_helpers.py" in os.listdir():
    print("Helpers schon vorhanden")
else:
    print("Helpers nicht vorhanden, starte Download...", end="")
    os.system("wget -O opal_mc_ml_helpers.py \"https://raw.githubusercontent.com/NTW-Muenster/opal-mc-ml/main/opal_mc_ml_helpers.py\"")
    print(" abgeschlossen!")
from opal_mc_ml_helpers import *
check_files()

Lade alle Events in eine Liste mit dem Namen eventliste

In [ ]:
eventliste = load_events()

Mit `eventliste[x]` kann auf das `x-1`-ste Ereignis zugegriffen werden. Die Methode `.show_image()` gibt das Ereignis grafisch aus.

In [ ]:
eventliste[53].show_image(show_category=True)

Mit `show_overview()` erhält man eine Übersicht über die Verteilung der Kategorien. Der erste Parameter gibt an, wie die Spalte beschriftet sein soll. Danach folgen ein oder mehrere Listen mit Bildateien.

In [ ]:
show_overview(["Roh"], eventliste)

Nun werden die Bildateien in zwei Datensätze aufgeteilt. `tv` ist kurz für Training+Validierung. 

In [ ]:
eventliste_tv, eventliste_test = split_events_random(eventliste, fraction_first_block=0.3)

Im nächsten Schritt werden die `tv`-Daten vervielfältigt, damit das Training einfacher wird. Alle Bilder werden verdreifacht. Dafür werden jeweils zwei Kopien des Bildes mit zufälligen Rotationen erstellt. Die Hälfte der Bilder wird zusätzlich zufällig gespiegelt. 
Um das Training der Leptonenzerfälle zu verbessern werden die Daten zu diesen Events zusätzlich noch ver-20-facht.

In [ ]:
faktor_alle = 3
faktor_leptonen = 20
eventliste_tv_vermehrt = augment_events(eventliste_tv, [faktor_alle, faktor_alle*faktor_leptonen, faktor_alle*faktor_leptonen, faktor_alle*faktor_leptonen])

Nun werden die `tv`-Daten in Training (`t`) und Validierung (`v`) unterteilt.

Abschließend wird erneut eine Übersicht ausgegeben.

In [ ]:
eventliste_training, eventliste_validierung = split_events_random(eventliste_tv_vermehrt, fraction_first_block=0.8)

show_overview(["Roh", "TV", "TV verm", "Train", "Vali", "Test"], eventliste, eventliste_tv, eventliste_tv_vermehrt, eventliste_training, eventliste_validierung, eventliste_test)

Die Trainingsdaten sind jetzt passend vorbereitet. Im nächsten Block wird ein Modell zum Lernen erstellt. Die Struktur wird einmal ausgegeben. In den letzten beiden Zeilen werden die Trainings- und Validierungsdaten geladen.

In [ ]:
modell = MLModel()
modell.load_structure_default()
modell.show_structure()
modell.load_training_eventlist(eventliste_training)
modell.load_validation_eventlist(eventliste_validierung)

Mit `train()` wird das Training des Modells gestartet. `count_epochs` gibt an, wie oft über alle Trainings- und Validierungsdaten gelaufen wird. Bei diesen Daten sind häufig Werte zwischen 3 und 6 sinnvoll.

In [ ]:
modell.train(count_epochs=3)

Die Lernkurve zeigt an, wie die Treffergenauigkeit sich im Laufe des Trainings verändert. Dabei wird die Treffergenauigkeit in Trainingsdaten und Validierungsdaten getrennt aufgetragen.

In [ ]:
modell.show_learning_curve()

Mit dem trainierten Modell können wir nun die Kategorien der Testdaten vorhersagen. Dabei wird die schon zugewiesene Kategorie der Testdaten ignoriert. Die Confusion Matrix ist ein Indikator dafür, wie gut die Vorhersage mit dem Modell funktioniert. Ein perfektes Modell hätte nur Einträge auf der Diagonalen: Jede Kategorie wird vom Modell so erkannt, wie sie vorher schon dem Bild zugeordnet war. Tatsächlich treten häufig einige falsche Vorhersagen auf. 

In [ ]:
eventliste_vorhersage = modell.predict(eventliste_test)
show_confusion_matrix(eventliste_test, eventliste_vorhersage)

Hier wird die Übersicht über die Verzweigungsverhältnisse noch um eine Spalte mit den vorhergesagten Daten erweitert.

In [ ]:
show_overview(["Roh", "TV", "TV verm", "Train", "Vali", "Test", "Vorh"], eventliste, eventliste_tv, eventliste_tv_vermehrt, eventliste_training, eventliste_validierung, eventliste_test, eventliste_vorhersage)

Man kann sich nun noch anschauen, welche Bilder falsch vorhergesagt wurden. Gelegentlich ist eine Einteilung auch für Menschen nicht eindeutig möglich.

In [ ]:
show_false_predictions(eventliste_test, eventliste_vorhersage, count=5)